In [1]:
"""Checking a portfolio for transaction updates since an AsAt time


Attributes
----------
AsAt
Transaction Portfolios
GetPortfolioChanges API
"""

'Checking a portfolio for transaction updates since an AsAt time\n\n\nAttributes\n----------\nAsAt\nTransaction Portfolios\nGetPortfolioChanges API\n'

# 1. Introduction

In this notebook, we show how you can call LUSID to check if there have been any new, amended, or canceled transactions since a given `AsAt` datetime. From a technical point-of-view this is a two-step process:

1. We use the [GetPortfolioChanges](https://www.lusid.com/docs/api/#operation/GetPortfolioChanges) endpoint to get a list of all portfolios in a `scope` which have changes after an `AsAt` date. 
2. We use [GetTransactions](https://www.lusid.com/docs/api/#operation/GetTransactions) to collect a list of these transactions.

> Note: We use <b>GetPortfolioChanges</b> here as we don't want to repeatedly call <b>GetTransactions</b> for every portfolio in a scope if there are no changes to be fetched.

The full workflow is outlined in the following diagram:

![GetPortfolioChanges1](img/get-portfolio-changes/get-portfolio-changes.gif)

# 2. Setup LUSID

In [2]:
# Import general purpose packages
import os
import json
from datetime import datetime, timedelta, date
import pytz
from time import sleep
from termcolor import colored

# Import lusid specific packages
import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.cocoon.utilities import create_scope_id

# Import data wrangling packages
import pandas as pd

pd.set_option("display.max_columns", None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

In [3]:
# Define the transaction portfolio API

txn_port_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
entities_api = api_factory.build(lusid.api.EntitiesApi)
portfolios_api = api_factory.build(lusid.api.PortfoliosApi)

In [4]:
# Load a mapping file for DataFrame headers for the get holdings response
with open(r"config/get_holdings_mapping.json") as mappings_file:
    get_holdings_json_mapping = json.load(mappings_file)

# Load a mapping file for DataFrame headers for the get transactions response
with open(r"config/get_transactions_mapping.json") as mappings_file:
    get_txn_json_mapping = json.load(mappings_file)

In [5]:
def datetime_to_filter_string(entry_datetime):
    
    date_obj = entry_datetime
    
    return date_obj.isoformat()[:-6] + "0Z"

# 3. Create function to capture changes

To start, we create a function which will be used to track changes. This function has two roles. Firstly, it reports new and amended transactions since an `AsAt`. Secondly, it keeps track of when the function is called. This allows us to manage the following workflow:

* @T: Get transactions entered since time T-1
* @T+1:  Get transactions entered since time T
* @T+2:  Get transactions entered since time T+1 

In [6]:
def get_latest_changes_entered_since_asat_date(scope, as_ats):
    
    """
    This function returns all the transactions posted into all portfolios in a scope since 
    an AsAt time provided by the user.
    
    There is two step process:
    1. Call the GetPortfolioChanges API to get the list of portfolios with changes
    2. Call the GetTransactions API to get details of transactions for a given portfolio
    
    """
    
    # The max_date resolves to the year 9999
    # We use this as a proxy to check for any changes across all effective space
    
    max_date = datetime.max.isoformat() + "Z"
    
    last_checked_date = as_ats[-1]
    
    last_checked_date_str = datetime_to_filter_string(last_checked_date)
    
    def get_transactions_since_date(portfolio_code, last_checked_date_str):
                
                
        api_filter = f"entryDateTime gt {last_checked_date_str}"
                                        
        get_transactions = txn_port_api.get_transactions(
            scope=scope,
            code=portfolio_code,
            filter=api_filter)
        
        txn_df = lusid_response_to_data_frame(get_transactions, rename_properties=True)
                
        return txn_df
    
    portfolios_with_changes =  [(i.entity_id.code, i.correction_as_at) for i in entities_api.get_portfolio_changes(
        scope=scope, effective_at=max_date, as_at=last_checked_date).values]
    
    for portfolio in portfolios_with_changes:
        
        print(f"Fetching transactions for portfolio {portfolio[0]} since AsAt time {last_checked_date_str}")
        
        txns_df = get_transactions_since_date(portfolio[0], last_checked_date_str)
        
        display(txns_df)
        
    latest_change_as_at_time = max([i[1] for i in portfolios_with_changes])
        
    as_ats.append(latest_change_as_at_time)

We use [GetPortfolioCommands](https://www.lusid.com/docs/api/#operation/GetPortfolioCommands) to see all commands since the beginning of the portfolio's life:

In [7]:
def list_commands_since_beginning_of_port_life(as_ats):
    
    latest_as_at = as_ats[-1]
    
    print("\n") 
    print(f"Rolling list of commands since creation date...")
     
    for portfolio in [portfolio_code1, portfolio_code2]:
        
        print("\n")                     
        print(f"\x1b[31mCommands for portfolio: {portfolio} \x1b[0m")
        
        commands = portfolios_api.get_portfolio_commands(scope=scope, code=portfolio).values

        for command in enumerate(commands, start=1):
            
            print(f"Command #{command[0]} {command[1].description}: {command[1].processed_time}")

# 4. Create two sample portfolio for demonstration

Here we create two portfolios for demonstration, each containing UK equities from the FTSE100. We upload the transactions in two batches, one for each portfolio.

In [8]:
# Create a new scope

scope = create_scope_id()
portfolio_code1 = "EQUITY_UK1"
portfolio_code2 = "EQUITY_UK2"

## 4.1 Create the portfolios in LUSID

In [9]:
portfolio_upsert_time = []

for code in [portfolio_code1, portfolio_code2]:

    try:

        response = txn_port_api.create_portfolio(
            scope=scope,
            create_transaction_portfolio_request=models.CreateTransactionPortfolioRequest(
                display_name=code,
                code=code,
                base_currency="GBP",
                created="2010-01-01",
                sub_holding_keys=[],
            ),
        )
                
        portfolio_upsert_time.append(response.version.as_at_date)

    except lusid.ApiException as e:
        print(json.loads(e.body)["title"])

We capture the `AsAt` time from the portfolio creation. This represents the first "event" in the worfklow. 

In [10]:
first_change = max(portfolio_upsert_time)

as_ats = [first_change]

## 4.2 Load transactions into these portfolios

Next, we load some transactions into LUSID. This will be another event.

In [11]:
# Load a file of equity transactions

transactions_file = r"data/get-portfolio-changes/equity_transactions.csv"
transactions_df = pd.read_csv(transactions_file)

In [12]:
transactions_df.head(2)

,portfolio_code,portfolio_name,portfolio_base_currency,ticker,sedol,instrument_type,instrument_id,name,txn_id,txn_type,txn_trade_date,txn_settle_date,txn_units,txn_price,txn_consideration,currency,strategy,cash_transactions
0,EQUITY_UK1,LUSID's top 10 FTSE stock portfolio,GBP,GB0002162385,SEDOL1,equity,EQ_1234,Aviva,trd_0001,Buy,02/01/2020,04/01/2020,120000,5,600000,GBP,ftse_tracker,NaN
1,EQUITY_UK1,LUSID's top 10 FTSE stock portfolio,GBP,GB00BGDT3G23,SEDOL8,equity,EQ_1241,Rightmove,trd_0015,Buy,15/01/2020,17/01/2020,80000,6,480000,GBP,ftse_tracker,NaN


In [13]:
# Load portfolios, instruments, and transactions

seed_data_response = seed_data(
    api_factory,
    ["instruments", "transactions"],
    scope,
    transactions_file,
    "csv",
)

## 4.3 Track changes since portfolio creation

Now lets use the function we defined above to show the transactions booked since portfolio creation.

In [14]:
get_latest_changes_entered_since_asat_date(scope, as_ats)

Fetching transactions for portfolio EQUITY_UK1 since AsAt time 2022-07-13T15:20:23.6810860Z


,transaction_id,type,instrument_identifiers.Instrument/default/ClientInternal,instrument_scope,instrument_uid,transaction_date,settlement_date,units,transaction_price.price,transaction_price.type,total_consideration.amount,total_consideration.currency,exchange_rate,transaction_currency,strategy(3ae4-ed37-7897-aa-Properties),SourcePortfolioId(default-Properties),SourcePortfolioScope(default-Properties),entry_date_time,transaction_status
0,trd_0001,Buy,EQ_1234,default,LUID_BM2Y3771,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,120000.0,5.0,Price,600000.0,GBP,1.0,GBP,ftse_tracker,EQUITY_UK1,3ae4-ed37-7897-aa,2022-07-13 15:20:26.082101+00:00,Active
1,trd_0015,Buy,EQ_1241,default,LUID_CNYYI8CU,2020-01-15 00:00:00+00:00,2020-01-17 00:00:00+00:00,80000.0,6.0,Price,480000.0,GBP,1.0,GBP,ftse_tracker,EQUITY_UK1,3ae4-ed37-7897-aa,2022-07-13 15:20:26.082101+00:00,Active


Fetching transactions for portfolio EQUITY_UK2 since AsAt time 2022-07-13T15:20:23.6810860Z


,transaction_id,type,instrument_identifiers.Instrument/default/ClientInternal,instrument_scope,instrument_uid,transaction_date,settlement_date,units,transaction_price.price,transaction_price.type,total_consideration.amount,total_consideration.currency,exchange_rate,transaction_currency,strategy(3ae4-ed37-7897-aa-Properties),SourcePortfolioId(default-Properties),SourcePortfolioScope(default-Properties),entry_date_time,transaction_status
0,trd_0016,Buy,EQ_1241,default,LUID_CNYYI8CU,2020-01-15 00:00:00+00:00,2020-01-17 00:00:00+00:00,80000.0,6.0,Price,480000.0,GBP,1.0,GBP,ftse_tracker,EQUITY_UK2,3ae4-ed37-7897-aa,2022-07-13 15:20:25.826383+00:00,Active
1,trd_0017,Buy,EQ_1242,default,LUID_O53ONFI9,2020-01-15 00:00:00+00:00,2020-01-17 00:00:00+00:00,450000.0,1.0,Price,450000.0,GBP,1.0,GBP,ftse_tracker,EQUITY_UK2,3ae4-ed37-7897-aa,2022-07-13 15:20:25.826383+00:00,Active


In [15]:
list_commands_since_beginning_of_port_life(as_ats)



Rolling list of commands since creation date...


Commands for portfolio: EQUITY_UK1 
Command #1 Create portfolio: 2022-07-13 15:20:23.412149+00:00
Command #2 Add transactions to portfolio: 2022-07-13 15:20:26.082101+00:00


Commands for portfolio: EQUITY_UK2 
Command #1 Create portfolio: 2022-07-13 15:20:23.681086+00:00
Command #2 Add transactions to portfolio: 2022-07-13 15:20:25.826383+00:00


# 5. Book a batch of fund flows for both portfolios

Next, we book a batch of fund flows into both funds. The first set of transactions are all backdated. We book some current and future dated transactions later.

In [16]:
# Define now in UTC
now = datetime.now(tz=pytz.UTC)

# Get SOD (midnight) today
today = date.today()
today_sod = pytz.utc.localize(datetime.combine(today, datetime.min.time()))

# Create upsert datetimes for yesterday
eight_pm_yesterday = today_sod - timedelta(hours=4)
ten_pm_yesterday = today_sod - timedelta(hours=2)

# Create upsert datetimes for today
nine_am_today = today_sod + timedelta(hours=9)
eleven_am_today = today_sod + timedelta(hours=11)

# Create upsert datetimes for tomorrow
nine_am_tomorrow = today_sod + timedelta(hours=33)

Define function to manage fund flow upsert:

In [17]:
def upsert_fund_flow_batch(fund_flow_txns):
    
    """
    This function upserts a set of fund flows into LUSID. 
    We purposefully leave 1 second between each upsert. 
    The function returns a the AsAt of last upsert
    
    """
    
    upsert_asat_times = []
    
    for portfolio_code in fund_flow_txns:
        
        sleep(1)
        
        transactions = fund_flow_txns[portfolio_code]
        
        upsert_transactions = txn_port_api.upsert_transactions(
            scope=scope,
            code=portfolio_code,
            transaction_request=[
                models.TransactionRequest(
                    transaction_id=transaction[0],
                    type=transaction[1],
                    instrument_identifiers={"Instrument/default/LusidInstrumentId": f"CCY_{transaction[2]}"},
                    transaction_date=transaction[3],
                    settlement_date=transaction[3],
                    units=transaction[4],
                    transaction_price=models.TransactionPrice(price=1, type="Price"),
                    total_consideration=models.CurrencyAndAmount(amount=transaction[4], currency=transaction[2]),
                    properties={},
                )
             for transaction in transactions]
        )
    
        upsert_asat_times.append(upsert_transactions.version.as_at_date)
                            
    return upsert_asat_times

In [18]:
fund_flow_txns_batch_1 = {
    
    # Post transactions for portfolio 1

    portfolio_code1: [ ("funds_001", "FundsIn", "GBP", eight_pm_yesterday, 100000),
    ("funds_002", "FundsIn", "USD", ten_pm_yesterday, 300000)],
    
    # Post transactions for portfolio 2

    portfolio_code2: [
            ("funds_001", "FundsIn", "GBP", eight_pm_yesterday, 100000),
    ("funds_002", "FundsIn", "USD", ten_pm_yesterday, 300000)
            
    ]
        
}

Upsert the fund flow transactions:

In [19]:
fund_flow_batch_1_upsert_asats = upsert_fund_flow_batch(fund_flow_txns_batch_1)

## 5.1 Track the transaction updates since fund flow posting


Once the fund flows have been upserted, we can run our function again to see the latest changes.


![GetPortfolioChanges2](img/get-portfolio-changes/get-changes-1.gif)


In [20]:
get_latest_changes_entered_since_asat_date(scope, as_ats)

Fetching transactions for portfolio EQUITY_UK2 since AsAt time 2022-07-13T15:20:26.0821010Z


,transaction_id,type,instrument_identifiers.Instrument/default/LusidInstrumentId,instrument_scope,instrument_uid,transaction_date,settlement_date,units,transaction_price.price,transaction_price.type,total_consideration.amount,total_consideration.currency,exchange_rate,transaction_currency,SourcePortfolioId(default-Properties),SourcePortfolioScope(default-Properties),entry_date_time,transaction_status
0,funds_001,FundsIn,CCY_GBP,default,CCY_GBP,2022-07-12 20:00:00+00:00,2022-07-12 20:00:00+00:00,100000.0,1.0,Price,100000.0,GBP,1.0,GBP,EQUITY_UK2,3ae4-ed37-7897-aa,2022-07-13 15:20:29.840198+00:00,Active
1,funds_002,FundsIn,CCY_USD,default,CCY_USD,2022-07-12 22:00:00+00:00,2022-07-12 22:00:00+00:00,300000.0,1.0,Price,300000.0,USD,1.0,USD,EQUITY_UK2,3ae4-ed37-7897-aa,2022-07-13 15:20:29.840198+00:00,Active


Fetching transactions for portfolio EQUITY_UK1 since AsAt time 2022-07-13T15:20:26.0821010Z


,transaction_id,type,instrument_identifiers.Instrument/default/LusidInstrumentId,instrument_scope,instrument_uid,transaction_date,settlement_date,units,transaction_price.price,transaction_price.type,total_consideration.amount,total_consideration.currency,exchange_rate,transaction_currency,SourcePortfolioId(default-Properties),SourcePortfolioScope(default-Properties),entry_date_time,transaction_status
0,funds_001,FundsIn,CCY_GBP,default,CCY_GBP,2022-07-12 20:00:00+00:00,2022-07-12 20:00:00+00:00,100000.0,1.0,Price,100000.0,GBP,1.0,GBP,EQUITY_UK1,3ae4-ed37-7897-aa,2022-07-13 15:20:28.325783+00:00,Active
1,funds_002,FundsIn,CCY_USD,default,CCY_USD,2022-07-12 22:00:00+00:00,2022-07-12 22:00:00+00:00,300000.0,1.0,Price,300000.0,USD,1.0,USD,EQUITY_UK1,3ae4-ed37-7897-aa,2022-07-13 15:20:28.325783+00:00,Active


In [21]:
list_commands_since_beginning_of_port_life(as_ats)



Rolling list of commands since creation date...


Commands for portfolio: EQUITY_UK1 
Command #1 Create portfolio: 2022-07-13 15:20:23.412149+00:00
Command #2 Add transactions to portfolio: 2022-07-13 15:20:26.082101+00:00
Command #3 Add transactions to portfolio: 2022-07-13 15:20:28.325783+00:00


Commands for portfolio: EQUITY_UK2 
Command #1 Create portfolio: 2022-07-13 15:20:23.681086+00:00
Command #2 Add transactions to portfolio: 2022-07-13 15:20:25.826383+00:00
Command #3 Add transactions to portfolio: 2022-07-13 15:20:29.840198+00:00


# 6. Post new and corrections to fund flow transactions

Finally, we post one last batch of transactions and a corrections. We add a correction here to illustrate that transactions are filtered by AsAt and not EffectiveAt. We also upsert transactions into one portfolio only, to demonstrate that [GetPortfolioChanges](https://www.lusid.com/docs/api/#operation/GetPortfolioChanges) only returns portfolios which actually have had changes.

In [22]:
fund_flow_txns_batch_2 = {
    
    # Post transactions for portfolio 2
    
    portfolio_code2: [
        
        ("funds_001", "FundsIn", "GBP", eight_pm_yesterday, 150000),
        ("funds_003", "FundsIn", "EUR", nine_am_today, 200000),
        ("funds_004", "FundsOut", "GBP", eleven_am_today, 200000),
        ("funds_005", "FundsIn", "GBP", nine_am_tomorrow, 200000)
    ]
    
}

In [23]:
fund_flow_upsert_2 = upsert_fund_flow_batch(fund_flow_txns_batch_2)

## 6.1 Get the latest transactions cut

Rerun the get changes function:

![GetPortfolioChanges3](img/get-portfolio-changes/get-changes-2.gif)


We can see below that the latest cut only includes:

1. New transactions
2. Corrections to old transactions posted since previous call

In [24]:
get_latest_changes_entered_since_asat_date(scope, as_ats)

Fetching transactions for portfolio EQUITY_UK2 since AsAt time 2022-07-13T15:20:29.8401980Z


,transaction_id,type,instrument_identifiers.Instrument/default/LusidInstrumentId,instrument_scope,instrument_uid,transaction_date,settlement_date,units,transaction_price.price,transaction_price.type,total_consideration.amount,total_consideration.currency,exchange_rate,transaction_currency,SourcePortfolioId(default-Properties),SourcePortfolioScope(default-Properties),entry_date_time,transaction_status
0,funds_001,FundsIn,CCY_GBP,default,CCY_GBP,2022-07-12 20:00:00+00:00,2022-07-12 20:00:00+00:00,150000.0,1.0,Price,150000.0,GBP,1.0,GBP,EQUITY_UK2,3ae4-ed37-7897-aa,2022-07-13 15:20:32.070259+00:00,Active
1,funds_003,FundsIn,CCY_EUR,default,CCY_EUR,2022-07-13 09:00:00+00:00,2022-07-13 09:00:00+00:00,200000.0,1.0,Price,200000.0,EUR,1.0,EUR,EQUITY_UK2,3ae4-ed37-7897-aa,2022-07-13 15:20:32.070259+00:00,Active
2,funds_004,FundsOut,CCY_GBP,default,CCY_GBP,2022-07-13 11:00:00+00:00,2022-07-13 11:00:00+00:00,200000.0,1.0,Price,200000.0,GBP,1.0,GBP,EQUITY_UK2,3ae4-ed37-7897-aa,2022-07-13 15:20:32.070259+00:00,Active
3,funds_005,FundsIn,CCY_GBP,default,CCY_GBP,2022-07-14 09:00:00+00:00,2022-07-14 09:00:00+00:00,200000.0,1.0,Price,200000.0,GBP,1.0,GBP,EQUITY_UK2,3ae4-ed37-7897-aa,2022-07-13 15:20:32.070259+00:00,Active


In [25]:
list_commands_since_beginning_of_port_life(as_ats)



Rolling list of commands since creation date...


Commands for portfolio: EQUITY_UK1 
Command #1 Create portfolio: 2022-07-13 15:20:23.412149+00:00
Command #2 Add transactions to portfolio: 2022-07-13 15:20:26.082101+00:00
Command #3 Add transactions to portfolio: 2022-07-13 15:20:28.325783+00:00


Commands for portfolio: EQUITY_UK2 
Command #1 Create portfolio: 2022-07-13 15:20:23.681086+00:00
Command #2 Add transactions to portfolio: 2022-07-13 15:20:25.826383+00:00
Command #3 Add transactions to portfolio: 2022-07-13 15:20:29.840198+00:00
Command #4 Add transactions to portfolio: 2022-07-13 15:20:32.070259+00:00


# 7. Clean-up: Delete portfolios

In [26]:
for port in (portfolio_code1, portfolio_code2):

    portfolios_api.delete_portfolio(scope=scope, code=port)